In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
# If code is in the drive, the following line works, otherwise add folder names
os.chdir('/content/drive/My Drive/Code/Data')

Mounted at /content/drive


In [ ]:
import pandas as pd
import os
import numpy as np
import re

import tensorflow as tf
from tensorflow.keras import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import EarlyStopping, Callback
import keras.backend as K

from tensorflow.keras.utils import plot_model

import pickle

In [ ]:
# Google colab test code
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
def reshape_data(X,y,ytrue=False):

  X = X.reshape(X.shape[0],X.shape[1],1)
  if ytrue==True:
    y = y.reshape(y.shape[0],1)
    return X,y

  return X

In [ ]:
%%time
# Load in OpenSubtitles training embeddings
with open('WTLOpenSubtitlesTrainEmbeddings.pkl', 'rb') as fp:
    oX_train = pickle.load(fp)

CPU times: user 210 ms, sys: 8.28 s, total: 8.49 s
Wall time: 1min 6s


In [ ]:
# Load in OpenSubtitles validation embeddings
with open('WTLOpenSubtitlesValidEmbeddings.pkl', 'rb') as fp:
    oX_valid = pickle.load(fp)

In [ ]:
# Load in OpenSubtitles ground truths
with open('YSWTLOpenSubtitlesTrainingSet.pkl', 'rb') as fp:

    oys = pickle.load(fp)

oy_train = oys[0]
oy_valid = oys[1]

In [ ]:
# Reshape to the right shape for the model
oX_train, oy_train = reshape_data(oX_train,oy_train,ytrue=True)
oX_valid, oy_valid = reshape_data(oX_valid,oy_valid,ytrue=True)

print(oX_train.shape, oy_train.shape, oX_valid.shape, oy_valid.shape)

(2613245, 768, 1) (2613245, 1) (365599, 768, 1) (365599, 1)


In [ ]:
# Visualization
oX_train

array([[[ 0.16443403],
        [ 0.06765132],
        [-0.01095102],
        ...,
        [ 0.01893931],
        [ 0.07692707],
        [-0.05215679]],

       [[ 0.05643836],
        [-0.07422301],
        [-0.01133703],
        ...,
        [ 0.16160056],
        [-0.09239875],
        [-0.0711588 ]],

       [[-0.06064596],
        [ 0.05612789],
        [-0.01445795],
        ...,
        [ 0.10499123],
        [-0.23381951],
        [-0.04740147]],

       ...,

       [[ 0.02480338],
        [ 0.06437417],
        [-0.00920172],
        ...,
        [ 0.07076002],
        [ 0.09588373],
        [-0.07820663]],

       [[-0.04667285],
        [-0.05105048],
        [-0.01126556],
        ...,
        [ 0.19108307],
        [-0.15953726],
        [-0.03212525]],

       [[-0.04684335],
        [ 0.16258712],
        [-0.00992162],
        ...,
        [ 0.3142661 ],
        [-0.06240847],
        [-0.10255329]]], dtype=float32)

In [ ]:
%%time
# Load in OpenSubtitles training embeddings
with open('DutchWTLOpenSubtitlesTrainEmbeddings.pkl', 'rb') as fp:
    odX_train = pickle.load(fp)

CPU times: user 247 ms, sys: 8.33 s, total: 8.58 s
Wall time: 56.6 s


In [ ]:
# Load in OpenSubtitles validation embeddings
with open('DutchWTLOpenSubtitlesValidEmbeddings.pkl', 'rb') as fp:
    odX_valid = pickle.load(fp)

In [ ]:
# Reshape to the right shape for the model
odX_train = reshape_data(odX_train,oy_train)
odX_valid = reshape_data(odX_valid,oy_valid)

print(odX_train.shape, oy_train.shape, odX_valid.shape, oy_valid.shape)

(2613245, 768, 1) (2613245, 1) (365599, 768, 1) (365599, 1)


## Heaton's TCN


In [ ]:
# REF: https://github.com/xjnpark/Deep-Learning-2023-t81/blob/master/t81_558_
def HeatonsTCN(X_train,y_train,X_valid,y_valid,loss):
  print('Build model...')
  model = Sequential()
  model = Sequential()
  model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
  model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
  model.add(Dropout(0.2))
  model.add(MaxPooling1D(pool_size=2))
  model.add(Flatten())
  model.add(Dense(50, activation='relu'))
  model.add(Dense(1))
  model.compile(loss=loss,
                optimizer='adam',metrics=['Accuracy','Precision','Recall','F1Score'])
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=50,
                          verbose=1, mode='auto', restore_best_weights=True)
  print('Train...')

  model.fit(X_train,y_train,validation_data=(X_valid,y_valid),
            callbacks=[monitor],verbose=2,epochs=12)#1000)

  return model, model.history

In [ ]:
%%time
# This one is set to 12 epochs instead of their 1000 because of the RAM
model = HeatonsTCN(oX_train, oy_train, oX_valid, oy_valid, 'binary_crossentropy')
model[1].history

Build model...
Train...
Epoch 1/12
90416/90416 - 411s - loss: 0.6557 - Accuracy: 0.6031 - precision: 0.6184 - recall: 0.5898 - f1_score: 0.6778 - val_loss: 0.6429 - val_Accuracy: 0.6181 - val_precision: 0.6059 - val_recall: 0.6723 - val_f1_score: 0.6660 - 411s/epoch - 5ms/step
Epoch 2/12
90416/90416 - 401s - loss: 0.6461 - Accuracy: 0.6147 - precision: 0.6293 - recall: 0.6042 - f1_score: 0.6778 - val_loss: 0.6446 - val_Accuracy: 0.6237 - val_precision: 0.6543 - val_recall: 0.5222 - val_f1_score: 0.6660 - 401s/epoch - 4ms/step
Epoch 3/12
90416/90416 - 401s - loss: 0.6437 - Accuracy: 0.6180 - precision: 0.6328 - recall: 0.6073 - f1_score: 0.6778 - val_loss: 0.6402 - val_Accuracy: 0.6256 - val_precision: 0.6216 - val_recall: 0.6395 - val_f1_score: 0.6660 - 401s/epoch - 4ms/step
Epoch 4/12
90416/90416 - 409s - loss: 0.6438 - Accuracy: 0.6181 - precision: 0.6327 - recall: 0.6079 - f1_score: 0.6778 - val_loss: 0.6425 - val_Accuracy: 0.6209 - val_precision: 0.6431 - val_recall: 0.5407 - val_f

{'loss': [0.6557204127311707,
  0.6460844278335571,
  0.6437356472015381,
  0.6438359618186951,
  0.6453314423561096,
  0.6440786123275757,
  0.6431350708007812,
  0.6425951719284058,
  0.6428109407424927,
  0.6421942114830017,
  0.6422460675239563,
  0.6435198187828064],
 'Accuracy': [0.603140652179718,
  0.6146949529647827,
  0.6180309653282166,
  0.6181114912033081,
  0.6161106824874878,
  0.6173393726348877,
  0.6195303201675415,
  0.6197490692138672,
  0.6201120018959045,
  0.6199948191642761,
  0.619942307472229,
  0.618392825126648],
 'precision': [0.6183600425720215,
  0.629342794418335,
  0.6327763199806213,
  0.6326936483383179,
  0.629699170589447,
  0.6305661201477051,
  0.6297085285186768,
  0.6274961233139038,
  0.6274241805076599,
  0.6266785264015198,
  0.6261091828346252,
  0.6247328519821167],
 'recall': [0.589832603931427,
  0.6041727066040039,
  0.6072675585746765,
  0.6079411506652832,
  0.6095579862594604,
  0.6121323108673096,
  0.6257165670394897,
  0.6353881359

## Keras TCN

In [ ]:
#pip install keras-tcn
!pip install keras-tcn --no-dependencies  # without the dependencies if you already have TF/Numpy.
from tcn import TCN, tcn_full_summary

In [ ]:
# https://github.com/philipperemy/keras-tcn/blob/master/tasks/sequential.py
def KerasTCN(X_train,y_train,X_valid,y_valid,loss):
  model = Sequential()
  model.add(TCN(
      nb_filters=64,
      kernel_size=6,
      dilations=[1, 2, 4, 8, 16, 32, 64]
  ))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  model.compile('adam', loss, metrics=['Accuracy','Precision','Recall','F1Score'])


  class TestCallback(Callback):

      def on_epoch_end(self, epoch, logs=None):
          print(logs)
          acc_key = 'val_Accuracy' if 'val_Accuracy' in logs else 'val_Acc'
#          assert logs[acc_key] > 0.78

          assert logs[acc_key] > 0.4

  print('Train...')
  model.fit(
      X_train, y_train,
      batch_size=10,
      validation_data=([X_valid, y_valid]),
      callbacks=[TestCallback()],epochs=10
  )
#40
  return model, model.history

In [ ]:
%%time
modelk = KerasTCN(oX_train, oy_train, oX_valid, oy_valid, 'binary_crossentropy')
modelk[1].history

Train...
Epoch 1/10
261325/261325 [==============================] - 3234s 12ms/step - loss: 0.6796 - Accuracy: 0.5555 - precision: 0.5592 - recall: 0.5980 - f1_score: 0.6746 - val_loss: 0.6836 - val_Accuracy: 0.5524 - val_precision: 0.5568 - val_recall: 0.5747 - val_f1_score: 0.6729
Epoch 2/10
261325/261325 [==============================] - 3203s 12ms/step - loss: 0.6777 - Accuracy: 0.5602 - precision: 0.5624 - recall: 0.6118 - f1_score: 0.6746 - val_loss: 0.6903 - val_Accuracy: 0.5521 - val_precision: 0.5600 - val_recall: 0.5441 - val_f1_score: 0.6729
Epoch 3/10
261325/261325 [==============================] - 3201s 12ms/step - loss: 0.6766 - Accuracy: 0.5625 - precision: 0.5629 - recall: 0.6283 - f1_score: 0.6747 - val_loss: 0.6820 - val_Accuracy: 0.5579 - val_precision: 0.5555 - val_recall: 0.6406 - val_f1_score: 0.6729
Epoch 4/10
261325/261325 [==============================] - 3196s 12ms/step - loss: 0.6766 - Accuracy: 0.5631 - precision: 0.5636 - recall: 0.6262 - f1_score: 0.67

{'loss': [0.6795547008514404,
  0.6776694059371948,
  0.6766185164451599,
  0.6766148805618286,
  0.6770855784416199,
  0.6786168813705444,
  0.6766764521598816,
  0.6798422932624817,
  0.677248477935791,
  0.676761269569397],
 'Accuracy': [0.5555311441421509,
  0.5601652264595032,
  0.5625454187393188,
  0.5630643367767334,
  0.5627237558364868,
  0.5628959536552429,
  0.5630034804344177,
  0.5630126595497131,
  0.5636306405067444,
  0.5635690689086914],
 'precision': [0.5592028498649597,
  0.5623800158500671,
  0.5628911852836609,
  0.5636417269706726,
  0.5634968876838684,
  0.5631074905395508,
  0.5626095533370972,
  0.5619004964828491,
  0.5622678399085999,
  0.5626223683357239],
 'recall': [0.5979518294334412,
  0.6117681264877319,
  0.6282778382301331,
  0.6262114644050598,
  0.6245090961456299,
  0.6294381022453308,
  0.6348325610160828,
  0.6413767337799072,
  0.6434926986694336,
  0.6397096514701843],
 'f1_score': [array([0.6746037], dtype=float32),
  array([0.6746401], dtype

# Experimental TCN

In [ ]:
# This proved to be the best model, this one is saved and a picture is created from it
def createandtrain4dilations(filter1,drop,loss,x,y,x2,y2):

  print(x.shape,y.shape,x2.shape,y2.shape)

  model = Sequential()
  model.add(Conv1D(filter1,kernel_size=3,activation='relu',dilation_rate=1))
  model.add(Conv1D(filter1,kernel_size=3,activation='relu',dilation_rate=2))
  model.add(Conv1D(64,kernel_size=3,activation='relu',dilation_rate=8))
  model.add(Conv1D(64,kernel_size=3,activation='relu',dilation_rate=32))
  model.add(Dropout(drop))
  model.add(AveragePooling1D())
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  monitor = EarlyStopping(monitor='val_loss',min_delta=1e-3, mode='auto', restore_best_weights=True)

  model.compile(loss=loss,
                optimizer='adam',
                metrics=['Accuracy','Precision','Recall','F1Score'])

  model.fit(x,y,validation_data=(x2,y2),verbose=0,epochs=15,callbacks=[monitor])#,batch_size=10)#0)

  # Save Keras model trained on English data
  #model.save('4dilationsTCN.keras')

  # Model trained on matrix multiplication of English and Dutch
  #model.save('EDosTCN.keras')
  """
  plot = tf.keras.utils.plot_model(
    model,
    to_file="KERASmodelTCNexonelayer.png",
    show_shapes=True,
    rankdir="LR",
    show_layer_names=False,
    expand_nested=True,
    show_layer_activations=True,
    show_trainable=False)
  """

  return model, model.history, model.predict(x[:50])

In [ ]:
def createandtrain3dilations(filter1,drop,loss,x,y,x2,y2):

  print(x.shape,y.shape,x2.shape,y2.shape)

  model = Sequential()
  model.add(Conv1D(filter1,kernel_size=3,activation='relu',dilation_rate=1))
  model.add(Conv1D(filter1,kernel_size=3,activation='relu',dilation_rate=2))
  model.add(Conv1D(64,kernel_size=3,activation='relu',dilation_rate=8))
  model.add(Dropout(drop))
  model.add(AveragePooling1D())
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  monitor = EarlyStopping(monitor='val_loss',min_delta=1e-3, mode='auto', restore_best_weights=True)

  model.compile(loss=loss,
                optimizer='adam',
                metrics=['Accuracy','Precision','Recall','F1Score'])

  model.fit(x,y,validation_data=(x2,y2),verbose=0,epochs=5,callbacks=[monitor],batch_size=10)#0)

  return model, model.history

In [ ]:
def createandtrain2dilations(filter1,drop,loss,x,y,x2,y2):

  print(x.shape,y.shape,x2.shape,y2.shape)

  model = Sequential()
  model.add(Conv1D(filter1,kernel_size=3,activation='relu',dilation_rate=1))
  model.add(Conv1D(64,kernel_size=3,activation='relu',dilation_rate=8))
  model.add(Dropout(drop))
  model.add(AveragePooling1D())
  model.add(Flatten())
  model.add(Dense(1, activation='sigmoid'))

  monitor = EarlyStopping(monitor='val_loss',min_delta=1e-3, mode='auto', restore_best_weights=True)

  model.compile(loss=loss,
                optimizer='adam',
                metrics=['Accuracy','Precision','Recall','F1Score'])

  model.fit(x,y,validation_data=(x2,y2),verbose=0,epochs=5,callbacks=[monitor],batch_size=10)#0)

  return model, model.history

In [ ]:
%%time
filter1= 128
drop = 0.5
modelL = createandtrain4dilations(filter1,drop,'binary_crossentropy',oX_train, oy_train, oX_valid, oy_valid)
print(modelL[1].history)

(2893287, 768, 1) (2893287, 1) (466101, 768, 1) (466101, 1)
{'loss': [0.6581066250801086, 0.6557859778404236], 'Accuracy': [0.5956208109855652, 0.5980561375617981], 'precision': [0.6060892343521118, 0.6082102656364441], 'recall': [0.6030925512313843, 0.6066917181015015], 'f1_score': [array([0.6777779], dtype=float32), array([0.67777807], dtype=float32)], 'val_loss': [0.6457333564758301, 0.649074375629425], 'val_Accuracy': [0.6114296913146973, 0.6102110743522644], 'val_precision': [0.6169173121452332, 0.6234302520751953], 'val_recall': [0.5848031044006348, 0.5536288022994995], 'val_f1_score': [array([0.66598266], dtype=float32), array([0.66598266], dtype=float32)]}
CPU times: user 59min 33s, sys: 6min 43s, total: 1h 6min 16s
Wall time: 45min 19s


In [ ]:
%%time
filter1= 32
drop = 0.1
modelL = createandtrain4dilations(filter1,drop,'binary_crossentropy',oX_train, oy_train, oX_valid, oy_valid)
print(modelL[1].history)

(2893287, 768, 1) (2893287, 1) (466101, 768, 1) (466101, 1)
{'loss': [0.6516103744506836, 0.6488934755325317], 'Accuracy': [0.6059430241584778, 0.6095022559165955], 'precision': [0.6171398758888245, 0.6201192736625671], 'recall': [0.6092013120651245, 0.6148819327354431], 'f1_score': [array([0.6777779], dtype=float32), array([0.6777779], dtype=float32)], 'val_loss': [0.6460970640182495, 0.6463671922683716], 'val_Accuracy': [0.6078575253486633, 0.6130537986755371], 'val_precision': [0.5916310548782349, 0.6038544774055481], 'val_recall': [0.6924991011619568, 0.6538771986961365], 'val_f1_score': [array([0.66598266], dtype=float32), array([0.66598266], dtype=float32)]}
CPU times: user 55min 37s, sys: 6min 51s, total: 1h 2min 28s
Wall time: 41min 28s


In [ ]:
%%time
filter1= 64
drop = 0.1
modelL = createandtrain3dilations(filter1,drop,'binary_crossentropy',oX_train, oy_train, oX_valid, oy_valid)
print(modelL[1].history)

(2613245, 768, 1) (2613245,) (365599, 768, 1) (365599,)
{'loss': [0.6768679618835449, 0.6748380064964294, 0.6746197938919067], 'Accuracy': [0.568280041217804, 0.5720627307891846, 0.5724805593490601], 'precision': [0.5731407403945923, 0.5765663981437683, 0.5768346190452576], 'recall': [0.5942229628562927, 0.5990158319473267, 0.6002843976020813], 'f1_score': [array([0.674532], dtype=float32), array([0.674532], dtype=float32), array([0.674532], dtype=float32)], 'val_loss': [0.6835342645645142, 0.6806802153587341, 0.6808470487594604], 'val_Accuracy': [0.5588198900222778, 0.5615825057029724, 0.5620092153549194], 'val_precision': [0.5528600811958313, 0.5687685608863831, 0.5773899555206299], 'val_recall': [0.679350733757019, 0.5597512125968933, 0.5080727338790894], 'val_f1_score': [array([0.67290413], dtype=float32), array([0.67290413], dtype=float32), array([0.67290413], dtype=float32)]}
CPU times: user 1h 16min 45s, sys: 9min 24s, total: 1h 26min 10s
Wall time: 55min 32s


In [ ]:
%%time
filter1= 64
drop = 0.5
modelL = createandtrain3dilations(filter1,drop,'binary_crossentropy',oX_train, oy_train, oX_valid, oy_valid)
print(modelL[1].history)

(2893287, 768, 1) (2893287, 1) (466101, 768, 1) (466101, 1)
{'loss': [0.6570326685905457, 0.6549206972122192], 'Accuracy': [0.5988790392875671, 0.6012552380561829], 'precision': [0.610491156578064, 0.6123521327972412], 'recall': [0.6008310914039612, 0.6053128838539124], 'f1_score': [array([0.6777779], dtype=float32), array([0.6777779], dtype=float32)], 'val_loss': [0.6446647047996521, 0.6439710855484009], 'val_Accuracy': [0.6134678721427917, 0.6143367886543274], 'val_precision': [0.626309871673584, 0.6167603135108948], 'val_recall': [0.5596797466278076, 0.6008199453353882], 'val_f1_score': [array([0.66598266], dtype=float32), array([0.66598266], dtype=float32)]}
CPU times: user 56min 9s, sys: 7min 22s, total: 1h 3min 31s
Wall time: 40min 28s


In [ ]:
%%time
filter1= 64
drop = 0.5
modelL = createandtrain4dilations(filter1,drop,'binary_crossentropy',oX_train, oy_train, oX_valid, oy_valid)
print(modelL[1].history)

(2893287, 768, 1) (2893287, 1) (466101, 768, 1) (466101, 1)
{'loss': [0.6573007702827454, 0.6568610668182373], 'Accuracy': [0.6031997799873352, 0.6033248901367188], 'precision': [0.6181674599647522, 0.6185057759284973], 'recall': [0.5909114480018616, 0.5901839137077332], 'f1_score': [array([0.6777779], dtype=float32), array([0.6777779], dtype=float32)], 'val_loss': [0.6500978469848633, 0.6503954529762268], 'val_Accuracy': [0.6175593137741089, 0.6167011260986328], 'val_precision': [0.6116562485694885, 0.6173093914985657], 'val_recall': [0.6407654881477356, 0.6110008358955383], 'val_f1_score': [array([0.66598266], dtype=float32), array([0.66598266], dtype=float32)]}
CPU times: user 43min 7s, sys: 6min 38s, total: 49min 46s
Wall time: 29min 11s


In [ ]:
%%time
filter1= 64
drop = 0.5

modelL = createandtrain4dilations(filter1,drop,'binary_crossentropy',oX_train, oy_train, oX_valid, oy_valid)

print(modelL[1].history,modelL[2])

(2613245, 768, 1) (2613245, 1) (365599, 768, 1) (365599, 1)
2/2 [==============================] - 0s 127ms/step
{'loss': [0.6778774857521057, 0.6755049824714661], 'Accuracy': [0.5650113224983215, 0.5681158900260925], 'precision': [0.5693669319152832, 0.571544885635376], 'recall': [0.5960644483566284, 0.6044975519180298], 'f1_score': [array([0.674532], dtype=float32), array([0.674532], dtype=float32)], 'val_loss': [0.6810640096664429, 0.6804027557373047], 'val_Accuracy': [0.5594490170478821, 0.5628024339675903], 'val_precision': [0.5588822364807129, 0.562622606754303], 'val_recall': [0.6224018931388855, 0.6188523769378662], 'val_f1_score': [array([0.67290413], dtype=float32), array([0.67290413], dtype=float32)]} [[0.55450755]
 [0.5380336 ]
 [0.486646  ]
 [0.58296955]
 [0.48047647]
 [0.44736385]
 [0.46351996]
 [0.5874745 ]
 [0.4211194 ]
 [0.46138567]
 [0.510892  ]
 [0.44308802]
 [0.5026344 ]
 [0.50216997]
 [0.5422502 ]
 [0.56425434]
 [0.5955512 ]
 [0.59065044]
 [0.47243026]
 [0.5186758 

## Train on Matrix Multiplications of Languages

In [ ]:
# Create matrix multiplications between languages
oED_train = np.multiply(oX_train, odX_train)
oED_valid = np.multiply(oX_valid, odX_valid)

In [ ]:
# Save matrix multiplication embeddings
#with open('EDembeddingsTrainValid.pkl', 'wb') as fp:
#    pickle.dump([oED_train,oED_valid], fp)
#    print('Saved')

Saved


In [ ]:
# Load in OpenSubtitles ground truths
with open('YSWTLOpenSubtitlesTrainingSet.pkl', 'rb') as fp:

    oys = pickle.load(fp)

oy_train = oys[0]
oy_valid = oys[1]

oX_train, oy_train = reshape_data(oX_train,oy_train,ytrue=True)

In [ ]:
with open('EDembeddingsTrainValid.pkl', 'rb') as fp:
    oED_train,oED_valid =  pickle.load(fp)

oED_train.shape

(2613245, 768, 1)

In [ ]:
%%time
filter1= 64
drop = 0.5

modelL = createandtrain4dilations(filter1,drop,'binary_crossentropy',oED_train, oy_train, oED_valid, oy_valid)

print(modelL[1].history,modelL[2])


(2613245, 768, 1) (2613245, 1) (365599, 768, 1) (365599, 1)
2/2 [==============================] - 0s 130ms/step
{'loss': [0.6808989644050598, 0.6791762709617615], 'Accuracy': [0.5584279298782349, 0.5606684684753418], 'precision': [0.5626102685928345, 0.5649769306182861], 'recall': [0.5944297313690186, 0.5943289995193481], 'f1_score': [array([0.674532], dtype=float32), array([0.674532], dtype=float32)], 'val_loss': [0.6835103034973145, 0.6833589673042297], 'val_Accuracy': [0.5549577474594116, 0.554098904132843], 'val_precision': [0.565424919128418, 0.5516242980957031], 'val_recall': [0.5284420251846313, 0.6443139910697937], 'val_f1_score': [array([0.67290413], dtype=float32), array([0.67290413], dtype=float32)]} [[0.52056944]
 [0.49696806]
 [0.48709738]
 [0.6150747 ]
 [0.5118736 ]
 [0.47518563]
 [0.4758995 ]
 [0.5509895 ]
 [0.47549152]
 [0.3888519 ]
 [0.38116476]
 [0.4525336 ]
 [0.52207506]
 [0.47935718]
 [0.4824239 ]
 [0.51934034]
 [0.5815374 ]
 [0.5404773 ]
 [0.44149917]
 [0.47595057

In [ ]:
%%time
filter1= 64
drop = 0.5

modelL = createandtrain4dilations(filter1,drop,'binary_crossentropy',oED_train, oy_train, oED_valid, oy_valid)

print(modelL[1].history,modelL[2])


(2613245, 768, 1) (2613245,) (365599, 768, 1) (365599,)
2/2 [==============================] - 1s 156ms/step
{'loss': [0.6827468872070312, 0.6817577481269836], 'Accuracy': [0.5541432499885559, 0.5552250146865845], 'precision': [0.5575191974639893, 0.5586926341056824], 'recall': [0.6003836393356323, 0.599736213684082], 'f1_score': [array([0.674532], dtype=float32), array([0.674532], dtype=float32)], 'val_loss': [0.6847208142280579, 0.6852788329124451], 'val_Accuracy': [0.5514046549797058, 0.5479609966278076], 'val_precision': [0.5599497556686401, 0.5482959747314453], 'val_recall': [0.5383947491645813, 0.6158477067947388], 'val_f1_score': [array([0.67290413], dtype=float32), array([0.67290413], dtype=float32)]} [[0.54896307]
 [0.47570512]
 [0.49591556]
 [0.6418487 ]
 [0.51308334]
 [0.51857024]
 [0.44250995]
 [0.59492534]
 [0.45665887]
 [0.4420201 ]
 [0.4043214 ]
 [0.46527785]
 [0.47211695]
 [0.4813898 ]
 [0.50457937]
 [0.48461723]
 [0.55924666]
 [0.5443207 ]
 [0.4593356 ]
 [0.5088605 ]
 